In [ ]:
from astropy.coordinates import SkyCoord

NAMES = [
    "CRTS J062029.9-250235",
    "CRTS J035520.3-484728",
    "Gaia DR3 2912314003686113664",
    "Gaia DR3 2900262462891972608",
    "CRTS J061939.7-244148",
]
coords = SkyCoord([SkyCoord.from_name(name) for name in NAMES])
coords
RADEC = list(zip(coords.ra.deg, coords.dec.deg))

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.time import Time

from approx_light_travel_time import fast_light_travel_time_heliocentric_elliptical

def add_helio_mjd(df):
    coord = SkyCoord(ra=df["diaObjectForcedSource.coord_ra"], dec=df["diaObjectForcedSource.coord_dec"], unit="deg")
    time = Time(df["diaObjectForcedSource.midpointMjdTai"], format="mjd", scale="tai")
    helio_time = time + fast_light_travel_time_heliocentric_elliptical(time, coord)
    df["diaObjectForcedSource.helioMjd"] = helio_time.mjd
    return df

In [ ]:
import lsdb

rows = []
for ra, dec in RADEC:
    cat = lsdb.open_catalog(
        "/sdf/data/rubin/user/kostya/hats/dia_object_lc_15k",
        search_filter=lsdb.ConeSearch(ra, dec, radius_arcsec=0.5),
    ).map_partitions(add_helio_mjd)
    df = cat.compute()
    assert len(df) == 1, f"{len(df) = }, {ra = }, {dec = }"
    row = df.iloc[0]
    rows.append(row)
rows

In [ ]:
lcs = [row["diaObjectForcedSource"].query(
    "~psfFlux_flag"
    " and ~psfDiffFlux_flag"
    " and ~pixelFlags_suspect"
    " and ~pixelFlags_saturated"
    " and ~pixelFlags_cr"
    " and ~pixelFlags_bad"
) for row in rows]
display(lcs[0])
display(lcs[1])

In [ ]:
import numpy as np
from astropy.timeseries import LombScargleMultiband

periods = []

for lc in lcs:
    model = LombScargleMultiband(bands=lc["band"], t=lc["helioMjd"], y=lc["psfMag"], dy=lc["psfMagErr"])
    freq, power = model.autopower(samples_per_peak=1000, nyquist_factor=10, minimum_frequency=1/0.7, maximum_frequency=1/0.1)
    periods.append(1.0 / freq[np.argmax(power)])
periods

In [ ]:
import matplotlib.pyplot as plt

COLORS = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00',
          'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

for name, row, lc, period in zip(NAMES, rows, lcs, periods):
    minimum_time = lc["helioMjd"][lc.query("band == 'r'")["psfMag"].idxmax()]
    lc["phase"] = (lc["helioMjd"] - minimum_time) % period / period

    plt.figure()
    for band in "ugrizy":
        data = lc.query("band == @band")
        label = band
        offset = 0.0
        if band == 'u':
            offset = -1.0
            label = f"{label}${offset:+.0f}$"
        plt.errorbar(data["phase"], data["psfMag"] + offset, yerr=data["psfMagErr"], label=label, fmt='o', color=COLORS[band])

    plt.gca().invert_yaxis()
    plt.xlabel("phase")
    plt.ylabel("PSF mag")
    plt.title(f"{name}\ndiaObjectID {row.diaObjectId}\nRA=${row.ra:.5f}$, Dec=${row.dec:.5f}$, P={period:.5f} d")
    plt.legend(loc='upper right')
    plt.savefig(f"{row.diaObjectId}.pdf", bbox_inches='tight')